<center><h1>VC04: Agrupamiento espectral, conceptos básicos</h1></center>

En esta práctica estudiaremos las ideas básicas que hemos visto como introducción al agrupamiento espectral.

Para empezar, cargamos las librerías que vamos a necesitar:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as it


Para comenzar, cargamos el conjunto de datos con el que trabajaremos:


In [ ]:
np.random.seed(17) # Fijamos una semilla para asegurar la reproducibilidad de la práctica

data_file_url = 'https://raw.githubusercontent.com/javier-sevilla/ANS/master/datasets/sinteticos/dataset_reducido.csv'
#data_file_url = 'https://raw.githubusercontent.com/javier-sevilla/ANS/master/datasets/sinteticos/dataset_circulos_concentricos.csv'
#data_file_url = 'https://raw.githubusercontent.com/javier-sevilla/ANS/master/datasets/sinteticos/dataset_cuatro_diferente_medida.csv'
#data_file_url = 'https://raw.githubusercontent.com/javier-sevilla/ANS/master/datasets/sinteticos/dataset_inseparable.csv'
#data_file_url = 'https://raw.githubusercontent.com/javier-sevilla/ANS/master/datasets/sinteticos/dataset_dos_remolinos.csv'
D = np.array(pd.read_csv(data_file_url,header=0))
D = D[ np.random.choice(np.arange(D.shape[0]), D.shape[0], replace=False),:]
Dx = D[:,:2]
#Dy = D[:,2]

fig, ax = plt.subplots(figsize=(10,5))
ax.scatter(Dx[:,0], Dx[:,1])#, c = Dy)



Antes de comenzar, cargamos la función que nos permite dibujar el grafo de afinidad:


In [ ]:
def plt_grafo_afinidad(Dx, A):
    minVal = np.min(A[np.nonzero(A)])
    aux = (A-minVal)/(np.max(A)-minVal)
    W = np.zeros(A.shape)
    W[np.nonzero(A)] = 5*(aux[np.nonzero(A)]+.1)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.scatter(Dx[:,0],Dx[:,1])
    inds = np.where(A>0)
    for i in np.arange(1,inds[0].size):
        ax.plot([Dx[inds[0][i],0],Dx[inds[1][i],0]],[Dx[inds[0][i],1],Dx[inds[1][i],1]], 
                linestyle='-', linewidth=W[inds[0][i],inds[1][i]],c='red')


En agrupamiento espectral trabajamos con la matriz de similitudes. Dada la matriz de distancias (euclidiana) que obtenemos con la funcion euclidean_distances de la librería scikit-learn, se calcula la matriz de similitud de la siguiente manera:


In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

sigma = 0.1

mSimilitud = euclidean_distances(Dx)
mSimilitud = np.exp(-np.power(mSimilitud,2)/(2*sigma**2))


Se puede construir un grafo de afinidad a partir de una matriz de similitud de diferentes maneras. Una de ellas es mediante el procedimiento de los vecinos más cercanos. Cada ejemplo se enlaza con sus $K$ vecinos más cercanos. A la hora de darle peso a cada arista, existen también diferentes apreciaciones. Una de las más sencillas sería asignar peso $1$ a la arista entre dos nodos $(e,f)$ si $e$ es uno de los $K$ vecinos más cercanos de $f$ y $f$ lo es de $e$. Se asignaría peso $0.5$ si sólo se cumpliese en un sentido.


In [ ]:
KNN = 4

# No queremos que un nodo sea vecino más cercano de si mismo
np.fill_diagonal(mSimilitud, 0)

W = np.zeros(mSimilitud.shape)
a = (np.argsort(-mSimilitud, axis=0)[0:KNN,:]).flatten()
b = np.tile(np.arange(mSimilitud.shape[0]),KNN)
W[a,b] = 1

a = np.repeat(np.arange(mSimilitud.shape[0]),KNN)
b = (np.argsort(-mSimilitud, axis=1)[:,0:KNN]).flatten()
W[a,b] += 1

W /= 2

plt_grafo_afinidad(Dx, W)


En la parte de teoría afirmábamos que un algoritmo de agrupamiento sobre un conjunto de datos puede verse como un algoritmo de corte sobre el grafo de afinidad correspondiente.

Para explicar qué es un algoritmo de corte es necesario explicar la función de corte. Ésta es una función que se define sobre un grafo y dos subgrupos complementarios de los nodos del grafo. La función corte cuenta el número de aristas que unen ambos subconjuntos de nodos. Si las aristas tienen un peso asignado, la función se define como la suma de los pesos de las aristas que unen ambos subconjuntos de nodos:


In [ ]:
def corte(W, A, B):
    return np.sum([[W[i,j] for j in B] for i in A])


Un algoritmo de corte busca en un grafo la separación en dos que minimiza la función de corte. Aunque existen aproximaciones eficientes (ver Algoritmo de Karger, https://en.wikipedia.org/wiki/Karger%27s_algorithm), la búsqueda del mejor corte por fuerza bruta se convierte rápidamente en inabarcable. Por eso, la limitamos a un número máximo de cortes 'stop_at':


In [ ]:
minCorte = np.inf
minAB = []
resCorteNorm = np.array([])
stop_at = 10000
for r in np.arange(1,W.shape[0]):
    for A in it.combinations(np.arange(W.shape[0]), r):        
        B = set(np.arange(W.shape[0]))-set(A)
        actCorte = corte(W, A, B)
        stop_at -= 1
        if actCorte < minCorte:
            minCorte = actCorte
            minAB = [set(A),B]
        if stop_at <= 0.:
            break

print('El valor de corte mínimo es: ', minCorte)
print('La separación que obtuvo corte mínimo es: ', minAB)

colores = np.zeros(Dx.shape[0])
colores[list(minAB[0])] = 1

fig, ax = plt.subplots(figsize=(10,5))
ax.scatter(Dx[:,0], Dx[:,1], c = colores)


Sin embargo, como se puede ver en el resultado anterior, por definición de la función de corte, este algoritmo básico tiende a devolver particiones extremas (ej., sólo un nodo en una de las particiones). Es por ello que en la práctica suele usarse la función de corte normalizado:


In [ ]:
def grado(W, i):
    return np.sum(W[i,:])

def corte_normalizado(W, G, A, B):
    normal = 1. / np.sum([G[i] for i in A]) + 1. / np.sum([G[j] for j in B])
    
    return normal * corte(W, A, B)


Se puede volver a hacer la prueba de la búsqueda del mejor corte por fuerza bruta (limitada a un número máximo de cortes 'stop_at'):


In [ ]:
D = np.array([grado(W,i) for i in np.arange(W.shape[0])])

minCorte = np.inf
minAB = []
resCorteNorm = np.array([])
stop_at = 10000
for r in np.arange(1,W.shape[0]):
    for A in it.combinations(np.arange(W.shape[0]), r):        
        B = set(np.arange(W.shape[0]))-set(A)
        # Una vez tenemos un posible corte A-B, calculamos el valor de corte normalizado
        actCorte = corte_normalizado(W, D, A, B)
        stop_at -= 1
        if actCorte < minCorte:
            minCorte = actCorte
            minAB = [set(A),B]
        if stop_at <= 0.:
            break

print('El valor de corte mínimo es: ', minCorte)

print('La separación que obtuvo corte mínimo es: ', minAB)

colores = np.zeros(Dx.shape[0])
colores[list(minAB[0])] = 1

fig, ax = plt.subplots(figsize=(10,5))
ax.scatter(Dx[:,0], Dx[:,1], c = colores)


Como puede observarse, el tamaño de las particiones está más compensado al usar la medida de corte normalizado ya que se tiene en cuenta el grado medio de los elementos de ambas particiones.


<hr>

En la teoría hablábamos también de la <b>matriz de transiciones</b> (probabilidad de saltar de un nodo a otro) en uno y en varios saltos. La multiplicación de la matriz de transición por sí misma $n$ veces devuelve la probabilidad de saltar de un nodo a otro en $n$ pasos. 

En primer lugar, definiremos la matriz de transiciones:

In [ ]:
mTrans = np.transpose(np.transpose(W)/np.sum(W,axis=0))
print('La matriz de transiciones para los dos primeros ejemplos:\n',mTrans[:2,:])


A continuación, calcularemos la matriz de transiciones en $n$ pasos. En este ejemplo, vamos a usar $n = 10$:


In [ ]:
mNTrans = mTrans.copy()
n = 10
for r in np.arange(n):
    mNTrans = mNTrans.dot(mTrans)


Podemos observar la probabilidad de acceder desde cada nodo al resto:


In [ ]:
for i in np.arange(Dx.shape[0]):
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(Dx[i,0], Dx[i,1], marker='*', markersize=10, color = 'y')
    ax.scatter(Dx[:,0], Dx[:,1], s = 200 * mNTrans[i,:])


¡Prueba con diferentes valores de $n$!
